In [3]:
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

# Define transformation to convert images to tensors
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize pixel values to the range [-1, 1]
])

# Load MNIST dataset
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

# Define parameters
input_size = 28 * 28  # Input size (28x28 pixels)
output_size = 10  # Output size (multi-class classification, 10 classes)
learning_rate = 0.01
epochs = 10
batch_size = 64

# Define sigmoid neuron model
class SigmoidNeuron(torch.nn.Module):
    def __init__(self, input_size):
        super(SigmoidNeuron, self).__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

# Initialize model
model = SigmoidNeuron(input_size)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()  # Cross Entropy Loss for multi-class classification
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Training loop
for epoch in range(epochs):
    for batch in train_loader:
        images, labels = batch
        images = images.view(-1, 28*28)  # Flatten images
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    # Print loss after each epoch
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, 28*28)  # Flatten images
        outputs = model(images)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get the predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')


Epoch [1/10], Loss: 1.7865
Epoch [2/10], Loss: 1.6422
Epoch [3/10], Loss: 1.6563
Epoch [4/10], Loss: 1.7230
Epoch [5/10], Loss: 1.6662
Epoch [6/10], Loss: 1.7030
Epoch [7/10], Loss: 1.6706
Epoch [8/10], Loss: 1.6590
Epoch [9/10], Loss: 1.5885
Epoch [10/10], Loss: 1.6680
Accuracy on the test set: 88.74%
